The Steps
---
1. [download new data, concatenate data](fbprophet-Introduction.ipynb)
   - [Dive new Features, US Change, etc](USChanges.ipynb)
   - [Noises ?]()
- [Forceast data for the next week, finalize data, and output result](GreyBox-ind2-xgb)
- [ToDo, pack models](...)


Reference
---
1. [Stock Prediction](Python-Machine-Learning-By-Example-master_chap07/Chapter07/1stock_price_prediction.py)
- [FFT prediction](Test-stock-prediction-algorithms-master/Misc%20experiments/FFT_Stock_Prediction.py)
- [Volatility](Test-stock-prediction-algorithms-master/Misc%20experiments/Volatility.py)

Note
---
1. [Data length](#Length-Calculation)
- [Prediction Vis](#Prediction-Visualization)


Data
---
1. data1 (orig), yahoo/20180518 (new),  yahoo/update20180518 (features added)
-  yahoo/20180518pre, yahoo/20180518pre30
-  yahoo//YahooFinance, (Market Index)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib,os
from stockstats import StockDataFrame as Sdf
# load Lasso
from sklearn.linear_model import  Lasso ,MultiTaskElasticNet
# load NN functions
#from keras.models import Sequential
#from keras.layers.core import Dense, Activation

from xgboost import XGBRegressor,plot_importance

from fbprophet import Prophet

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
USind={'^IXIC':'NDAQ','^RUT':'Russel2000','^GSPC':'SandP','GC=F':'Gold','CL=F':'Oil','^VIX':'VIX','^DJA':'DJA','^DJI':'DJI',
       'DX-Y.NYB':'USDInx'}
US_indecies=['^IXIC','^RUT','^GSPC','GC=F','CL=F','^VIX','^DJA','^DJI','DX-Y.NYB']

In [ ]:
# create US Stock market dataframe, df_yahoo

df_yahoo = pd.DataFrame()
for US_index in US_indecies:
    dfname="yahoo/YahooFinance/%s.csv" %USind[US_index]
    featurename='%s' %USind[US_index]
    #print(dfname)
    df_tmp=pd.read_csv(dfname,index_col='Date') 
    df_yahoo[featurename]=df_tmp.Close
    
    del df_tmp

In [ ]:
df0050_date=pd.read_csv("yahoo/20180518pre/0050.csv",index_col='Date') 
df0050_date.info()

In [ ]:
df0050_date_1=df0050_date.copy()
stock_df = Sdf.retype(df0050_date_1)
#df0050_date['rsi5']=stock_df['rsi_5']

In [ ]:
df0050_date_1.info()

Volatility
---

1. $\mathbf{\triangle x_i= \log x_i -\log x_{i-1}}$, the return of $x_i$
-  Volatility=$\mathbf{\sum_w \sigma(\triangle x_i)/w}$, where the window, $w$, is the length of considered period, and $\sigma$ is the (positive standard deviation.

**Robert L. McDonald**, Derivatives Markets (3rd Edition) (Pearson Series in Finance)


**Note**

If no data from US market, comment out features creation df_US

In [ ]:
def generate_features_p23(df):
    """ Generate features for a stock/index based on historical price and performance
    Args:
        df (dataframe with columns "Open", "Close", "High", "Low", "Volume", "Adjusted Close")
    Returns:
        dataframe, data set with new features
    """
    df_new = pd.DataFrame()
    # 6 original features
    df_new['High'] = df['High']
    df_new['Low'] = df['Low']
    df.Date=df.index
    df_new['weekday']=pd.DatetimeIndex(df.Date).dayofweek.values
    df_new['weekday']=df_new['weekday'].astype('int')
    #df_new['weekday'] = df['weekday']
    #df_new['code'] = df['code']
    #df_new['price_change'] = df['price_Change']
    #df_new['sign'] = df['sign']
    
    
    df_new['Open'] = df['Open']
    df_new['open_1'] = df['Open'].shift(1)
    df_new['close_1'] = df['Close'].shift(1)
    df_new['high_1'] = df['High'].shift(1)
    df_new['low_1'] = df['Low'].shift(1)
    df_new['volume_1'] = df['Volume'].shift(1)
    
    # 31 original features

    # rolling_mean desprecated since pandas 0.23
    # use df.rolling(windows).mean().shift(1) in placed
    #      df.rolling(windows).std().shift(1)
    
        # average price
    df_new['avg_price_5'] = df['Close'].rolling(5).mean().shift(1)
    df_new['avg_price_30'] = df['Close'].rolling(21).mean().shift(1)
    df_new['avg_price_365'] = df['Close'].rolling(252).mean().shift(1)
    df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
    df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
    df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']
    
    
    # average volume
    df_new['avg_volume_5'] = df['Volume'].rolling(5).mean().shift(1)
    df_new['avg_volume_30'] = df['Volume'].rolling(21).mean().shift(1)
    df_new['avg_volume_365'] = df['Volume'].rolling(252).mean().shift(1)
    df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
    df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
    df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']
    
    # standard deviation of prices
    
    df_new['std_price_5'] = df['Close'].rolling(5).std().shift(1)
    df_new['std_price_30'] = df['Close'].rolling(21).std().shift(1)
    df_new['std_price_365'] = df['Close'].rolling(252).std().shift(1)
    df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
    df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
    df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']
    
    # standard deviation of volumes
    df_new['std_volume_5'] = df['Volume'].rolling(5).std().shift(1)
    df_new['std_volume_30'] = df['Volume'].rolling(21).std().shift(1)
    df_new['std_volume_365'] = df['Volume'].rolling(252).std().shift(1)
    df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
    df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
    df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']
    # # return
    df_new['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
    df_new['return_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
    df_new['return_30'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
    df_new['return_365'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)
    
    df_new['return_1'].rolling(5).mean()
    df_new['moving_avg_5'] = df_new['return_1'].rolling(5).mean()
    df_new['moving_avg_30'] =df_new['return_1'].rolling(21).mean()
    df_new['moving_avg_365'] = df_new['return_1'].rolling(252).mean()
    
    df_new['Log Ret'] = np.log(df['Close']/df['Close'].shift(1))
    w = 63 # ~ 1 year
    df_new['VolatilityQuater'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    w = 6
    df_new['VolatilityWeek'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    w=28
    df_new['VolatilityMonth'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    #w=252
    #df_new['VolatilityYear'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    
    df_US=pd.read_csv("USChange.csv",index_col='Date') 
    df_new['US']=df_US.Close
    
    df_US['yesterday']=0
    df_US['yesterday'][1:]=df_US.Close[1:]
    df_US['US_Change']=df_US.Close-df_US.yesterday
    df_new['US_Change']=df_US.US_Change
    
    del df_US
    
    #RSI
       
    df_1=df.copy()
    stock_df = Sdf.retype(df_1)
    ## reset Index 
    #stock_df.index=pd.to_datetime(stock_df.index, format='%Y%m%d')
    #stock_df.index.names = ['Date']
    
    #print(stock_df['rsi_6'].tail())
    df_new['RSI6']=stock_df['rsi_6'].values
    df_new['RSI12']=stock_df['rsi_12'].values
    df_new['RSI24']=stock_df['rsi_24'].values
    df_new['VR']=stock_df['vr'].values
    df_new['VR_6_sma']=stock_df['vr_6_sma'].values
    df_new['volume_delta']=stock_df['volume_delta'].values
    df_new['open_2_d']=stock_df['open_2_d'].values
    df_new['open_-2_r']=stock_df['open_-2_r'].values
    df_new['cr']=stock_df['cr'].values
    df_new['cr-ma1']=stock_df['cr-ma1'].values
    df_new['cr-ma2']=stock_df['cr-ma2'].values
    df_new['cr-ma3']=stock_df['cr-ma3'].values
    df_new['kdjk']=stock_df['kdjk'].values
    df_new['kdjd']=stock_df['kdjd'].values
    df_new['kdjj']=stock_df['kdjj'].values
    df_new['macd']=stock_df['macd'].values
    
    # data from US stock market
    
    for US_index in US_indecies:       
        featurename='%s' %USind[US_index]
        df_name="yahoo/YahooFinance/%s.csv" %USind[US_index]
        df_yahoo=pd.read_csv(df_name,index_col='Date') 
        df_new[featurename]=df_yahoo['Close']
    
    
    
    # the target
    df_new['Close'] = df['Close']
    df_new=df_new.fillna(method='ffill')
    df_new=df_new.fillna(method='bfill')
    #df_new = df_new.dropna(axis=0)
    return df_new

In [ ]:
def generate_features_p23_1(df):
    """ Generate features for a stock/index based on historical price and performance
    Args:
        df (dataframe with columns "Open", "Close", "High", "Low", "Volume", "Adjusted Close")
    Returns:
        dataframe, data set with new features
    """
    df_new = pd.DataFrame()
    # 6 original features
    df_new['High'] = df['High']
    df_new['Low'] = df['Low']
    df.Date=df.index
    df_new['weekday']=pd.DatetimeIndex(df.Date).dayofweek.values
    df_new['weekday']=df_new['weekday'].astype('int')
    #df_new['weekday'] = df['weekday']
    #df_new['code'] = df['code']
    #df_new['price_change'] = df['price_Change']
    #df_new['sign'] = df['sign']
    
    
    df_new['Open'] = df['Open']
    df_new['open_1'] = df['Open'].shift(1)
    df_new['close_1'] = df['Close'].shift(1)
    df_new['high_1'] = df['High'].shift(1)
    df_new['low_1'] = df['Low'].shift(1)
    df_new['volume_1'] = df['Volume'].shift(1)
    
    # 31 original features

    # rolling_mean desprecated since pandas 0.23
    # use df.rolling(windows).mean().shift(1) in placed
    #      df.rolling(windows).std().shift(1)
    
        # average price
    df_new['avg_price_5'] = df['Close'].rolling(5).mean().shift(1)
    df_new['avg_price_30'] = df['Close'].rolling(21).mean().shift(1)
    #df_new['avg_price_365'] = df['Close'].rolling(252).mean().shift(1)
    df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
    #df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
    #df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']
    
    
    # average volume
    df_new['avg_volume_5'] = df['Volume'].rolling(5).mean().shift(1)
    df_new['avg_volume_30'] = df['Volume'].rolling(21).mean().shift(1)
    #df_new['avg_volume_365'] = df['Volume'].rolling(252).mean().shift(1)
    df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
    #df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
    #df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']
    
    # standard deviation of prices
    
    df_new['std_price_5'] = df['Close'].rolling(5).std().shift(1)
    df_new['std_price_30'] = df['Close'].rolling(21).std().shift(1)
    #df_new['std_price_365'] = df['Close'].rolling(252).std().shift(1)
    df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
    #df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
    #df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']
    
    # standard deviation of volumes
    df_new['std_volume_5'] = df['Volume'].rolling(5).std().shift(1)
    df_new['std_volume_30'] = df['Volume'].rolling(21).std().shift(1)
    #df_new['std_volume_365'] = df['Volume'].rolling(252).std().shift(1)
    df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
    #df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
    #df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']
    # # return
    df_new['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
    df_new['return_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
    df_new['return_30'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
    #df_new['return_365'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)
    
    df_new['return_1'].rolling(5).mean()
    df_new['moving_avg_5'] = df_new['return_1'].rolling(5).mean()
    df_new['moving_avg_30'] =df_new['return_1'].rolling(21).mean()
    #df_new['moving_avg_365'] = df_new['return_1'].rolling(252).mean()
    
    df_new['Log Ret'] = np.log(df['Close']/df['Close'].shift(1))
    w = 63 # ~ 1 year
    df_new['VolatilityQuater'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    w = 6
    df_new['VolatilityWeek'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    w=28
    df_new['VolatilityMonth'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    #w=252
    #df_new['VolatilityYear'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    
    df_US=pd.read_csv("USChange.csv",index_col='Date') 
    df_new['US']=df_US.Close
    
    df_US['yesterday']=0
    df_US['yesterday'][1:]=df_US.Close[1:]
    df_US['US_Change']=df_US.Close-df_US.yesterday
    df_new['US_Change']=df_US.US_Change
    
    del df_US
    
    #RSI
       
    df_1=df.copy()
    stock_df = Sdf.retype(df_1)
    ## reset Index 
    #stock_df.index=pd.to_datetime(stock_df.index, format='%Y%m%d')
    #stock_df.index.names = ['Date']
    
    #print(stock_df['rsi_6'].tail())
    df_new['RSI6']=stock_df['rsi_6'].values
    df_new['RSI12']=stock_df['rsi_12'].values
    df_new['RSI24']=stock_df['rsi_24'].values
    df_new['VR']=stock_df['vr'].values
    df_new['VR_6_sma']=stock_df['vr_6_sma'].values
    df_new['volume_delta']=stock_df['volume_delta'].values
    df_new['open_2_d']=stock_df['open_2_d'].values
    df_new['open_-2_r']=stock_df['open_-2_r'].values
    df_new['cr']=stock_df['cr'].values
    df_new['cr-ma1']=stock_df['cr-ma1'].values
    df_new['cr-ma2']=stock_df['cr-ma2'].values
    df_new['cr-ma3']=stock_df['cr-ma3'].values
    df_new['kdjk']=stock_df['kdjk'].values
    df_new['kdjd']=stock_df['kdjd'].values
    df_new['kdjj']=stock_df['kdjj'].values
    df_new['macd']=stock_df['macd'].values
    
    # data from US stock market
    
    for US_index in US_indecies:       
        featurename='%s' %USind[US_index]
        df_name="yahoo/YahooFinance/%s.csv" %USind[US_index]
        df_yahoo=pd.read_csv(df_name,index_col='Date') 
        df_new[featurename]=df_yahoo['Close']
    
    
    
    # the target
    df_new['Close'] = df['Close']
    df_new=df_new.fillna(method='ffill')
    df_new=df_new.fillna(method='bfill')
    df_new = df_new.dropna(axis=0)
    return df_new

In [ ]:
data=generate_features_p23_1(df0050_date)

data=generate_features(df_taihua)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


In [ ]:
X_columns = list(data.drop(['Close'], axis=1).columns)
y_column = 'Close'

In [ ]:
def gridsceme(df,size=150):
    
    
    data=generate_features_p23(df0050_date)
    X_columns = list(data.drop(['Close'], axis=1).columns)
    y_column = 'Close'
    size=size
    X = data[X_columns][-size:]
    y = data[y_column][-size:]
    
    val_ratio =0.2
    X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=val_ratio,random_state=42)

In [ ]:
#X=data.as_matrix()
#Y=data.Close
size=150
X = data[X_columns][-150:]
y = data[y_column][-150:]

In [ ]:

val_ratio =0.2

X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=val_ratio,random_state=42)
#X_test=X[-5:]
#Y_test=y[-5:]

Evaluation
---
1. $\mathbf{MSE}=\frac{1}{n}\sqrt{\sum{(y_i-y^{prec}_i)^2}}$: small (~0) for good, big for bad (~1)
- $\mathbf{MAE}=\frac{1}{n}{\sum{\left|y_i-y^{prec}_i\right|}}$: small (~0) for good, big for bad (~1)
- $\mathbf{R^2}$: small (~0) for bad, big for good (~1)
  - $\mathbf{SST=\text{Sum of tatal error}}=\sum(y_i-\bar y)^2$
  - $\mathbf{SSR=\text{Sum of Residues}}=\sum(y_i-y^{prec}_i)^2$
  - $\mathbf{R^2=1-\frac{SSR}{SST}}$, 

In [ ]:
from sklearn.model_selection import GridSearchCV
# First experiment with linear regression

# SGD is very sensitive to data with features at different scales. Hence we need to do feature scaling before training.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_val = scaler.transform(X_val)

param_grid = {
    "alpha": [1e-5, 3e-5, 1e-4],
    "eta0": [0.01, 0.03, 0.1],
}

from sklearn.linear_model import SGDRegressor
lr = SGDRegressor(penalty='l2', n_iter=1000)
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_scaled_train, y_train)

print(grid_search.best_params_)

lr_best = grid_search.best_estimator_
# print(grid_search.best_score_)

predictions_lr = lr_best.predict(X_scaled_val)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_lr)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_lr)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_lr)))


In [ ]:
X_scaled_test = scaler.transform(X_test)
lr_best.predict(X_scaled_test)

In [ ]:
df0050_date=pd.read_csv("yahoo/20180518/0050.csv",index_col='Date') 

def gridsceme(df,size=150):
    
    
    data=generate_features_p23(df0050_date)
    X_columns = list(data.drop(['Close'], axis=1).columns)
    y_column = 'Close'
    size=size
    X = data[X_columns][-size:]
    y = data[y_column][-size:]
    
    val_ratio =0.2
    X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=val_ratio,random_state=42)
    
    param_grid = {
       "learning_rate": [0.05,0.1,0.2],
       "max_depth": [5, 10, 15, 20],
       "colsample_bytree": [0.7,0.8,0.9],
    }

    model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

    grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)

    print(grid_search.best_params_)
    # print(grid_search.best_score_)

    xgb_best = grid_search.best_estimator_
    predictions_xgb = xgb_best.predict(X_val)

    print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_xgb)))
    print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_xgb)))
    print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_xgb)))



In [ ]:
# Next experiment with random forest

param_grid = {
    "max_depth": [30, 50],
    "min_samples_split": [5, 10, 20],

}

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
# print(grid_search.best_score_)

rf_best = grid_search.best_estimator_
predictions_rf = rf_best.predict(X_val)

print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_rf)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_rf)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_rf)))


In [ ]:
rf_best.predict(X_test)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

param_grid = {
    "learning_rate": [0.05,0.1,0.2],
    "max_depth": [5, 10, 15, 20],
    "colsample_bytree": [0.7,0.8,0.9],
}


model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
# print(grid_search.best_score_)

xgb_best = grid_search.best_estimator_
predictions_xgb = xgb_best.predict(X_val)

print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_xgb)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_xgb)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_xgb)))


In [ ]:
xgb_best.predict(X_test)

In [ ]:
 xgb_best.predict(X_test)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

param_grid = {
    "learning_rate": [0.05,0.1,0.2],
    "max_depth": [5, 10, 15, 20],
    "colsample_bytree": [0.7,0.8,0.9],
}

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_val = scaler.transform(X_val)


model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_scaled_train, y_train)

print(grid_search.best_params_)
# print(grid_search.best_score_)

xgb_best = grid_search.best_estimator_
predictions_xgb = xgb_best.predict(X_scaled_val)

print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_xgb)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_xgb)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_xgb)))


In [ ]:
X_scaled_test = scaler.transform(X_test)
xgb_best.predict(X_scaled_test)

In [ ]:
model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

The Weight of Features
---

Create a new pandas DataFrame with two features, X_train.columns and xgb_best.feature_importances_. By basic pandas operations, we can sketch out what the important features in the mode:

In [ ]:
df_p=pd.DataFrame()
df_p['features']=X_train.columns
df_p['importance']=xgb_best.feature_importances_

df_pp=df_p[df_p.importance>0.01]
df_pp=df_pp.sort_values(['importance'],ascending=True)

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
n=len(df_pp)
plt.yticks(list(range(n)), df_pp.features, fontsize=18)
plt.barh(df_pp.features,df_pp.importance)
plt.title('0050', fontsize=18)

function of Importance Visualization
---

In [ ]:
def plotImportance(xgb_best,features,etf='0050'):
    df_p=pd.DataFrame()
    df_p['features']=features
    df_p['importance']=xgb_best.feature_importances_

    df_pp=df_p[df_p.importance>0.01]
    df_pp=df_pp.sort_values(['importance'],ascending=True)
    fig,ax = plt.subplots(figsize=(15,15))
    n=len(df_pp)
    plt.yticks(list(range(n)), df_pp.features, fontsize=18)
    plt.barh(df_pp.features,df_pp.importance)
    plt.title(etf)

In [ ]:
features=X_train.columns
plotImportance(xgb_best,features)

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
ax.set_yticklabels(X_train.columns)

plot_importance(xgb_best,height=0.5,ax=ax,max_num_features=20)

In [ ]:
df0050_date=pd.read_csv("yahoo/update20180518/00692.csv",index_col='Date') 


In [ ]:
len(df0050_date)

In [ ]:
def gridsceme(df,size=150,etf='0050'):
    if (len(df)>365):    
       data=generate_features_p23(df)
    else:
       data=generate_features_p23_1(df) 
    X_columns = list(data.drop(['Close'], axis=1).columns)
    y_column = 'Close'
    #size=size
    X = data[X_columns][-size:]
    y = data[y_column][-size:]
    #print(len(data))
    val_ratio =0.2
    X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=val_ratio,random_state=42)
    
    param_grid = {
       "learning_rate": [0.05,0.1,0.2],
       "max_depth": [5, 10, 15, 20],
       "colsample_bytree": [0.7,0.8,0.9],
    }

    model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

    grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)

    print(grid_search.best_params_)
    # print(grid_search.best_score_)

    xgb_best = grid_search.best_estimator_
    predictions_xgb = xgb_best.predict(X_val)

    print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_xgb)))
    print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_xgb)))
    print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_xgb)))

    features=X_train.columns
    plotImportance(xgb_best,features,etf=etf)

In [ ]:
etfs=['0050','0051','0052','0053','0054','0055','0056','0057','0058',
      '0059','006201','006203','006204','006208','00690','00692','00701','00713']

In [ ]:
datadir="yahoo/update20180518"

for etf in  etfs:
    
    etf_name=datadir+"/%s.csv" %etf
    df_tmp=pd.read_csv(etf_name,index_col='Date')
    print(etf,'\n----') 
    #df_tmp.info()
    gridsceme(df_tmp,size=150,etf=etf)
    del df_tmp

In [ ]:
df0050_date.tail()

In [ ]:
df0050_date.info()

In [ ]:
import seaborn; seaborn.set()

In [ ]:
gridsceme(df0050_date,size=150)

In [ ]:
model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

In [ ]:
model_xgb.fit(X, Y)

In [ ]:
Y_pred_xgb=model_xgb.predict(X)
Y_pred_xgb[-5:]

Length Calculation
---

In [ ]:
import os

dirname="yahoo/update20180518/" 

print("Summary of ETF's\n...\n")
for file in os.listdir(dirname):
    print("ETF-%s" %file[:-4])
    df=pd.read_csv(dirname+file)
    l=len(df)
    day1=df.Date[0]
    day2=df.Date[l-1]
    print(l," days include: begin from ",day1," to ",day2)
    del df

In [ ]:
import os
from datetime import datetime
dirname="yahoo/update20180518/" 

print("Summary of ETF's\n...\n")
print('{:10}{:12}{:13}{:6}'.format("ETF_code","Start_Date","End_Date"," Days"))
for file in os.listdir(dirname):
    #print("ETF-%s" %file[:-4])
    name=file[:-4]
    df=pd.read_csv(dirname+file)
    l=len(df)
    day1=df.Date[0]
    day2=df.Date[l-1]
    print('{:1}{:9}{:12}{:12}{:6}'.format(" ",name,day1,day2,l))
    del df

Prediction Visualization
---

In [ ]:
weekpre="yahoo/20180518pre30/"

plt.figure(figsize=(16,18))

i=1
for file in os.listdir(weekpre):
    f=pd.read_csv(weekpre+file)
    plt.subplot(6,3,i)
    s="%s" %file[:-3]
    plt.plot(f.Close[-30:],label=s) 
    plt.legend(loc='upper right')
    i=i+1
#plt.legend(True)   